# Notebook de prise en main du package dedupe

Ce notebook vise à présenter le fonctionnement du package dedupe et les possibilités qu'il offre. Le package dedupe est d'abord un outil dédié au dédoublonnage, mais l'appariement de deux fichiers est aussi pris en compte. Il adopte une approche ambitieuse reprosant sur du _machine learning_.

Les données utilisées sont des données synthétiques issues du [générateur de données synthétiques du package FEBRL](https://users.cecs.anu.edu.au/~Peter.Christen/Febrl/febrl-0.3/febrldoc-0.3/node70.html). Deux fichiers sont disponibles. Le premier fichier contient 5000 individus fictifs dont les traits d'identité (nom, prénom, date de naissance, etc.) proviennent de tables de fréquence tirés d'annuaires téléphoniques australiens. Le second fichier contient les mêmes 5000 individus, mais avec des erreurs sur les traits d'identité (substitution de caractères, suppression d'une valeur, remplacement par des variantes communes d'orthographe, etc.).

La documentation du package dedupe est disponible [ici](https://docs.dedupe.io/en/latest/).

Les différents termes techniques et notions évoquées dans ce notebook sont définis dans le document de travail associé.

## Import des librairies et chargement des données

In [1]:
### Installation des packages recordLinkage et dedupe
!pip install recordLinkage
!pip install dedupe

In [2]:
### Import des librairies nécessaires
import pandas as pd
import recordlinkage
import dedupe
from recordlinkage.datasets import load_febrl4

In [3]:
### Chargement des données
dfA, dfB = load_febrl4()

In [5]:
n = len(dfA)

Nous avons chargé les données sous forme de DataFrame de la librairie pandas. C'est un format pratique pour manipuler des données tabulaires.

In [6]:
dfA.head()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-1070-org,michaela,neumann,8,stanley street,miami,winston hills,4223,nsw,19151111,5304218
rec-1016-org,courtney,painter,12,pinkerton circuit,bega flats,richlands,4560,vic,19161214,4066625
rec-4405-org,charles,green,38,salkauskas crescent,kela,dapto,4566,nsw,19480930,4365168
rec-1288-org,vanessa,parr,905,macquoid place,broadbridge manor,south grafton,2135,sa,19951119,9239102
rec-3585-org,mikayla,malloney,37,randwick road,avalind,hoppers crossing,4552,vic,19860208,7207688


Le package dedupe prend en entrée des dictionnaires, on procède donc à une transformation des données. 

In [7]:
### Création de dictionnaires pour coller au format requis par le package
dfA_dict = dfA.astype(str).to_dict(orient = 'index')
dfB_dict = dfB.astype(str).to_dict(orient = 'index')

In [8]:
### Affichage de quelques éléments du dictionnaire
print(dfA_dict['rec-1070-org'])
print(dfA_dict['rec-1016-org'])
print(dfA_dict['rec-4405-org'])

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(dfA_dict['rec-1070-org'], dfA_dict['rec-1016-org'], dfA_dict['rec-4405-org'])? (2599590402.py, line 2)

## Appariement

Un appariement avec dedupe repose sur du _machine learning_, et plus particulièrement sur un processus d'___active learning___. Concrètement, il va être nécessaire d'annoter un échantillon de paires pour que l'algorithme puisse par la suite apprendre à les classer. L'approche proposée par le package dedupe est intéressante parce qu'elle vise à maximiser l'information obtenue de l'annotation pour un nombre de paires annotées donné.
L'_active learning_ consiste à faire intervenir le modèle dans le choix des paires à annoter. Le modèle va proposer à l'annotation les paires sur lesquelles il est le plus incertain, ce qui permet d'améliorer plus rapidement la qualité des prédictions qu'avec des paires tirées aléatoirement.

L'autre particularité de l'approche du package dedupe concerne l'étape d'indexation. Ici, ce n'est pas l'utilisateur qui va définir des règles, **c'est l'algorithme qui essaie de déterminer automatiquement les meilleures règles de blocage**.

Le package a un côté "boîte noire" sur certaines parties de l'appariement. Ainsi, le déroulement classique n'est pas forcément respecté.

De façon pratique, un processus d'appariement avec le package dedupe se déroule de la façon suivante :
- Déclaration des champs identifiants
- Préparation de l'apprentissage
- Annotation par active learning
- Entraînement du modèle de classfication
- Prédiction du statut de chaque paire
- Évaluation de la qualité

### Déclaration des champs identifiants

La première étape consiste à définir quels champs vont être utilisés pour apparier les données, à déclarer leur type et la présence éventuelle de données manquantes. Pour cela, on vérifie d'abord quels champs contiennent des valeurs manquantes.

In [9]:
dfA.isna().sum()

given_name       112
surname           48
street_number    158
address_1         98
address_2        420
suburb            55
postcode           0
state             50
date_of_birth     94
soc_sec_id         0
dtype: int64

Dans ce notebook, on ne va utiliser qu'une partie des champs disponibles.

In [10]:
### Déclaration des champs identifiants ainsi que de leur type
fields = [{'field' : 'given_name', 'type': 'String', 'has missing':True},
          {'field' : 'surname', 'type': 'String', 'has missing':True},
          {'field' : 'address_1', 'type': 'String', 'has missing':True},
          {'field' : 'suburb', 'type': 'String', 'has missing':True},
          {'field' : 'postcode', 'type': 'String'},
          {'field' : 'date_of_birth', 'type': 'String', 'has missing':True},
         ]

On peut ensuite initialiser un objet que l'on nomme _linker_, sur lequel on va passer toutes les instructions suivantes.

In [11]:
### Initialisation de l'objet dedupe
linker = dedupe.RecordLink(fields)

### Préparation de l'apprentissage

La méthode _prepare_training_ permet de préparer l'objet linker pour le processus d'annotation. L'algorithme teste de premières règles de blocage sur une taille d'échantillon définie par le paramètre `sample_size`. Le temps de calcul croît avec la valeur de ce paramètre.

In [12]:
### Initialisation de l'active learner
linker.prepare_training(dfA_dict, dfB_dict, sample_size=500)

### Annotation par active learning

À ce stade, il est temps d'annoter des paires. Le modèle affiche des paires d'individus et l'utilisateur doit décider s'il s'agit du même individu ou non. Il n'y a pas de règle absolue sur le nombre de paires à annoter, mais la documentation du package conseille au moins 10 exemples positifs et 10 exemples négatifs. En pratique, il peut arriver que le modèle ne propose que des exemples négatifs ou inversement que des exemples positifs pendant assez longtemps...

Les annotations vont être stockées dans l'objet _linker_.

In [13]:
### Début du processus d'annotation par active learning
dedupe.console_label(linker)

given_name : julia
surname : clarke
address_1 : marconi crescent
suburb : whitfield
postcode : 2642
date_of_birth : nan

given_name : julia
surname : clarke
address_1 : marconi crescent
suburb : whitfield
postcode : 2642
date_of_birth : nan

0/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished


 y


given_name : hayden
surname : stapley
address_1 : tindale street
suburb : cromer heights
postcode : 4125
date_of_birth : nan

given_name : hayden
surname : stapley
address_1 : tindale street
suburb : cromer heights
postcode : 4125
date_of_birth : nan

1/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


 y


given_name : isobel
surname : campbell
address_1 : nan
suburb : strathalbyn
postcode : 2770
date_of_birth : 19470921

given_name : isobel
surname : campbell
address_1 : nan
suburb : strathalbyn
postcode : 2770
date_of_birth : 19470921

2/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


...

given_name : natalia
surname : maczkowiack
address_1 : badcoe street
suburb : south turramurra
postcode : 3199
date_of_birth : 19830221

given_name : nan
surname : mainsbridge
address_1 : cazaly close
suburb : colac
postcode : 3199
date_of_birth : 19110409

16/10 positive, 9/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


 n


given_name : joshua
surname : nguyen
address_1 : beattie crescent
suburb : casino
postcode : 3029
date_of_birth : 19240622

given_name : jessiac
surname : bastiaans
address_1 : beazley crescent
suburb : ingleburn
postcode : 3029
date_of_birth : 19270912

16/10 positive, 10/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


 f


Finished labeling


### Entraînement du modèle de classification

Une fois un nombre satisfaisant de paires annotées, on peut procéder à l'entraînement du modèle. L'algorithme va affiner les règles de blocage, puis entraîner une régression logistique pénalisée.
Les règles de blocage sont définies de façon à limiter au maximum le nombre de paires conservées, tout en conservant la plus grande proportion possible de paires annotées positivement dans l'étape précédente.

L'utilisateur peut jouer sur le paramètre `recall`, qui correspond à la proportion de paires annotées positivement qui seront conservées après application des règles de blocage. Une valeur proche de 0 pour ce paramètre va conduire à des règles de blocage assez strictes (on va retenir peu de paires), et inversement pour une valeur proche de 1. 

In [14]:
### Entraînement du modèle de classification (régression logistique)
linker.train(recall = 0.8)

### Prédiction du statut de chaque paire

Finalement, on peut demander au modèle de prédire le statut de chaque paire. En pratique, on obtient une liste contenant l'ensemble des paires liées par le modèle, ainsi que la probabilité associée. Par défaut, le modèle ne lie une paire que si la probabilité associée dépasse 0.5. Une étape de résolution des conflits est automatiquement mise en œuvre, de façon à ce qu'un individu du premier fichier soit apparié à au plus un individu du second fichier, et inversement. Si cette contrainte ne correspond pas aux fichiers à apparier (par exemple si l'un des deux fichiers contient des doublons), il est possible de changer la méthode de résolution des conflits via le paramètre `constraint`.

Par ailleurs, fixer le paramètre `threshold` à 0 permet de récupérer toutes les paires (après résolution des conflits), ce qui laisse ensuite la possibilité de les analyser plus en détail avant de décider du seuil.

In [15]:
### Prédiction du statut de chaque paire
linked_records = linker.join(dfA_dict, dfB_dict, threshold = 0.5)
print('Nombre de paires liées : ', len(linked_records))

Nombre de paires liées :  4699


In [16]:
### Affichage des 10 paires les plus probables
linked_records[:10]

[(('rec-999-org', 'rec-999-dup-0'), 0.93820375),
 (('rec-996-org', 'rec-996-dup-0'), 0.93820375),
 (('rec-991-org', 'rec-991-dup-0'), 0.93820375),
 (('rec-976-org', 'rec-976-dup-0'), 0.93820375),
 (('rec-975-org', 'rec-975-dup-0'), 0.93820375),
 (('rec-97-org', 'rec-97-dup-0'), 0.93820375),
 (('rec-960-org', 'rec-960-dup-0'), 0.93820375),
 (('rec-929-org', 'rec-929-dup-0'), 0.93820375),
 (('rec-927-org', 'rec-927-dup-0'), 0.93820375),
 (('rec-915-org', 'rec-915-dup-0'), 0.93820375)]

### Évaluation de la qualité

Évaluer la qualité d'un appariement est une étape primordiale mais souvent difficile à réaliser car on ne connaît pas le vrai statut de chaque paire. Il faut donc utiliser un échantillon représentatif annoté. (Attention, l'échantillon annoté via le processus d'_active learning_ n'est a priori pas représentatif de l'ensemble des paires. Évaluer les résultats sur cet échantillon conduirait sans doute à une estimation biaisée.)

Ici, la tâche est plus simple puisque nous travaillons avec des données générées. On peut donc calculer les résultats sur l'ensemble des individus.

Le package dedupe ne propose pas de fonction pour évaluer les résultats. Il faut donc dans un premier temps convertir la liste des paires liées en un DataFrame pandas et procéder à quelques manipulations pour calculer les mesures de qualité classiques.

In [17]:
def clean_results_dedupe(result):
    """
    Formats the output from dedupe to match the output from the record
    linkage package.

            Parameters:
                    result (list): output from the linkage with dedupe.
                        It is a list of pairs to be linked, with the
                        associated predicted match probability.

            Returns:
                    cleaned_results (pandas DataFrame): Predicted matches
    """
    
    #Create dataframe from list of tuples
    cleaned_results = pd.DataFrame(result, columns = ['id', 'classification'])
    
    #Split the id in two columns : rec_id_1 and rec_id_2
    cleaned_results[['rec_id_1', 'rec_id_2']] = pd.DataFrame(cleaned_results['id'].tolist())
    
    #Remove the id column
    cleaned_results.drop('id', axis=1, inplace=True)
    
    # Transform the probability into a predicted match status. Dedupe
    # outputs only matches so the predicted match status is always 1.
    cleaned_results['classification'] = 1
    
    return cleaned_results

def compute_performance_metrics_FEBRL(linkage_output, dataset_size):
    """
    Compute performance metrics of a record linkage process on FEBRL synthetic data.
    The assumption is that the size of the two datasets is the same and every record 
    from dataset A has exactly one match in dataset B.

            Parameters:
                    linkage output (pandas Series): Output from the linkage process
                    dataset_size (int): Length of both datasets to be linked

            Returns:
                    performance_metrics (tuple): Tuple of metrics (TP, TN, FP, FN, precision, recall, F-measure)
    """
    linkage_output = linkage_output.reset_index()
    linkage_output['actual'] = (linkage_output['rec_id_1'].str.extract(r'(rec-\d+)') 
                                == linkage_output['rec_id_2'].str.extract(r'(rec-\d+)'))
    
    TP = sum(linkage_output['classification'] & linkage_output['actual'])
    FP = sum(linkage_output['classification'] & ~linkage_output['actual'])
    #Pairs that were removed in the indexing phase must be taken into account to compute True and False negatives
    FN = dataset_size - TP
    TN = dataset_size*dataset_size - dataset_size - FP
    
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    Fscore = 2 * precision * recall / (precision + recall)
    performance_metrics = (TP, TN, FP, FN, precision, recall, Fscore)
    return(performance_metrics)
    
def print_performance_metrics(linkage_output, dataset_size):
    """
    Prints performance metrics of a record linkage process on synthetic data.
    The assumption is that the size of the two datasets is the same and every record 
    from dataset A has exactly one match in dataset B.

            Parameters:
                    linkage output (pandas Series): Output from the linkage process
                    dataset_size (int): Length of both datasets to be linked

            Returns:
                    None
    """
    TP, TN, FP, FN, precision, recall, Fscore = compute_performance_metrics_FEBRL(linkage_output, dataset_size)
    print(f"Vrais positifs : {TP:,}".replace(',', ' '))
    print(f"Vrais négatifs : {TN:,}".replace(',', ' '))
    print(f"Faux positifs : {FP:,}".replace(',', ' '))
    print(f"Faux négatifs : {FN:,}".replace(',', ' '))
    print(f"Précision : {precision:.4}")
    print(f"Rappel : {recall:.4}")
    print(f"F-mesure : {Fscore:.4}")

In [18]:
cleaned_results = clean_results_dedupe(linked_records)
print_performance_metrics(cleaned_results, 5000)

Vrais positifs : 4 699
Vrais négatifs : 24 995 000
Faux positifs : 0
Faux négatifs : 301
Précision : 1.0
Rappel : 0.9398
F-mesure : 0.969


Les résultats semblent satisfaisants. En particulier, il n'y a aucun faux positif ou presque (cela peut dépendre des choix effectués lors de l'annotation), probablement dû au fait que les erreurs générées sur le fichier B sont assez légères et ne perturbent donc pas trop l'algorithme d'appariement.

Quoi qu'il en soit, ces résultats sont propres aux deux fichiers appariés. Un autre jeu de fichiers, avec des données de qualité différente et des types d'erreurs différents, donnerait un tout autre résultat.

Voici un exemple de paire que le modèle a manquée. Il a particulièrement de mal avec les inversions de champs.

In [33]:
dfA_dict['rec-992-org']

{'given_name': 'nikita',
 'surname': 'mcgregor',
 'street_number': '1',
 'address_1': 'sherwood circuit',
 'address_2': 'cooranga',
 'suburb': 'frankston',
 'postcode': '3892',
 'state': 'sa',
 'date_of_birth': '19250127',
 'soc_sec_id': '5134549'}

In [32]:
dfB_dict['rec-992-dup-0']

{'given_name': 'mcgregor',
 'surname': 'nikita',
 'street_number': '1',
 'address_1': 'cooranlga',
 'address_2': 'sherwood ircuit',
 'suburb': 'frankston',
 'postcode': '3892',
 'state': 'sa',
 'date_of_birth': '19250127',
 'soc_sec_id': '9742298'}

## Exemple de code complet pour un appariement

In [23]:
### Import des librairies nécessaires
import pandas as pd
import recordlinkage
import dedupe
from recordlinkage.datasets import load_febrl4

### Chargement des données
dfA, dfB = load_febrl4()

### Création de dictionnaires pour coller au format requis par le package
dfA_dict = dfA.astype(str).to_dict(orient = 'index')
dfB_dict = dfB.astype(str).to_dict(orient = 'index')

### Déclaration des champs identifiants ainsi que de leur type
fields = [{'field' : 'given_name', 'type': 'String', 'has missing':True},
          {'field' : 'surname', 'type': 'String', 'has missing':True},
          {'field' : 'address_1', 'type': 'String', 'has missing':True},
          {'field' : 'suburb', 'type': 'String', 'has missing':True},
          {'field' : 'postcode', 'type': 'String'},
          {'field' : 'date_of_birth', 'type': 'String', 'has missing':True},
         ]

### Initialisation de l'objet dedupe
linker = dedupe.RecordLink(fields)

### Initialisation de l'active learner
linker.prepare_training(dfA_dict, dfB_dict, sample_size=500)

### Début du processus d'annotation par active learning
dedupe.console_label(linker)

### Entraînement du modèle de classification (régression logistique)
linker.train(recall = 0.8)

### Prédiction du statut de chaque paire
linked_records = linker.join(dfA_dict, dfB_dict, threshold = 0.5)

given_name : hayden
surname : stapley
address_1 : tindale street
suburb : cromer heights
postcode : 4125
date_of_birth : nan

given_name : hayden
surname : stapley
address_1 : tindale street
suburb : cromer heights
postcode : 4125
date_of_birth : nan

0/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished


 y


given_name : kirra
surname : browne
address_1 : shoalhaven avenue
suburb : surry hills
postcode : 3042
date_of_birth : 19430407

given_name : kirra
surname : browne
address_1 : shoalhaven avenue
suburb : surry hills
postcode : 3042
date_of_birth : 19403407

1/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


 y


given_name : samuel
surname : princehorn
address_1 : gould street
suburb : warra
postcode : 3152
date_of_birth : 19761209

given_name : samuel
surname : princehorn
address_1 : gould street
suburb : warra
postcode : 3152
date_of_birth : 19716209

2/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


...

given_name : joel
surname : scheerlinck
address_1 : boswell crescent
suburb : bicton
postcode : 2290
date_of_birth : 19521024

given_name : nan
surname : noble
address_1 : costello circuit
suburb : bicton
postcode : 2290
date_of_birth : 19710327

23/10 positive, 2/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


 n


given_name : nan
surname : fullerton
address_1 : leichhardt street
suburb : chelsea heights
postcode : 6107
date_of_birth : 19620113

given_name : olivia
surname : mcnnll
address_1 : alberga street
suburb : lue
postcode : 6107
date_of_birth : 19100419

23/10 positive, 3/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


 f


Finished labeling
/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
